In [1]:
import numpy as np
import pandas as pd
import warnings
from time import sleep
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")

from cyber_sdk.key.mnemonic import MnemonicKey

from src.utils_bash import get_json_from_bash_query
from src.utils_cyber import create_subgraph_links
from config import BOSTROM_LCD_CLIENT, BOSTROM_NODE_RPC_URL, WALLET_SEED, ipfs_client

In [2]:
mk = MnemonicKey(mnemonic=WALLET_SEED)
WALLET = BOSTROM_LCD_CLIENT.wallet(mk)
CONTENT_TYPE_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1823qj9q6eruxv8mfwfay87zd9pp66ayq0ckp9xttk0e758a6kl9qtglw7z'
TYPE_CONTENT_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1pc4ep362vkyrquq7r27wvm08u2yjxwnuylc509s2dfumw4qwt7hsdavx33'

## Get data for types of particles

In [3]:
content_type_df = pd.read_csv('data/particles_with_filetype_20230418.csv')[['particle', 'type']]
content_type_to_cid_dict = {_content_type: ipfs_client.add_str(_content_type) for _content_type in list(content_type_df.type.unique())}
content_type_df['type_cid'] = content_type_df['type'].map(lambda content_type: content_type_to_cid_dict[content_type])
content_type_df.head()

,particle,type,type_cid
0,QmfNfxY5AuE5rrDnE8w8TH7n9fPJQwLhvCo8xHELrP657i,audio,QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
1,QmV4wJWemDkVLcsXxpgwPvMnNNdUdV1NzN3ZgwaUuVbStM,audio,QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
2,QmdbvwcPfckUzFtcsnvBj28pMua5feXg5k6W1nn5nDeeVT,audio,QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
3,QmScFB5z8Z9ijWR9TkRJXB7NSHUSsR21z1MURbYMie9ZDr,audio,QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX
4,QmYmbqTjkmfZHDDt15X8WW1xwPmx5eKDjCz5w4k9uF7d6C,audio,QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX


# Create cyberlinks by the subgraph contract

In [4]:
def get_remained_bandwidth(address: str = CONTENT_TYPE_SUBGRAPH_CONTRACT_ADDRESS, node_rpc_url: str = BOSTROM_NODE_RPC_URL, cli_name: str = 'cyber') -> int:
    bandwidth_json = get_json_from_bash_query(f'{cli_name} query bandwidth neuron {address} -o=json --node {node_rpc_url}')
    return int(bandwidth_json['neuron_bandwidth']['remained_value'])

### Content -> Type Subgraph

In [5]:
print(f'Content -> Type Subgraph contract remained bandwidth links {get_remained_bandwidth(address=CONTENT_TYPE_SUBGRAPH_CONTRACT_ADDRESS)//250:>,}')

Content -> Type Subgraph contract remained bandwidth links 14,400


In [8]:
txs_broadcasted = []
links_broadcasted = []
links_unsuccessful_broadcasted = []
for content_type_item_df in tqdm(np.array_split(content_type_df.iloc[21000:], indices_or_sections=100)):
    links_list = []
    for _, row in content_type_item_df.iterrows():
        links_list.append([row.particle, row.type_cid])

    tx_broadcasted = create_subgraph_links(
        link_candidates=links_list, wallet=WALLET, subgraph_contract_address=CONTENT_TYPE_SUBGRAPH_CONTRACT_ADDRESS,
        memo='upload content types github.com/Snedashkovsky/knowledge_graph_research')
    txs_broadcasted.append(tx_broadcasted)
    if tx_broadcasted:
        links_broadcasted.append(links_list)
    else:
        links_unsuccessful_broadcasted.append(links_list)
        sleep(30)
    sleep(2)

### Type -> Content Subgraph

In [9]:
print(f'Type -> Content Subgraph contract remained bandwidth links {get_remained_bandwidth(address=TYPE_CONTENT_SUBGRAPH_CONTRACT_ADDRESS)//250:>,}')

Type -> Content Subgraph contract remained bandwidth links 26


In [10]:
txs_broadcasted = []
links_broadcasted = []
links_unsuccessful_broadcasted = []
for content_type_item_df in tqdm(np.array_split(content_type_df.iloc[10:20], indices_or_sections=1)):
    links_list = []
    for _, row in content_type_item_df.iterrows():
        links_list.append([row.type_cid, row.particle])

    tx_broadcasted = create_subgraph_links(
        link_candidates=links_list, wallet=WALLET, subgraph_contract_address=TYPE_CONTENT_SUBGRAPH_CONTRACT_ADDRESS,
        memo='upload content types github.com/Snedashkovsky/knowledge_graph_research')
    txs_broadcasted.append(tx_broadcasted)
    if tx_broadcasted:
        links_broadcasted.append(links_list)
    else:
        links_unsuccessful_broadcasted.append(links_list)
        sleep(30)
    sleep(2)

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
links_broadcasted

[[['QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX',
   'QmY6dsuGeLLQMdseCgWCh4Uu3keK5iXXfo8wfUUWM5SRMR'],
  ['QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX',
   'QmbYwLb7gxZumzFfMJTgBaThuZ9Bs247EhVTAZ4srAhe5b'],
  ['QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX',
   'QmVSb9zX5aiHwQDCGoCpmAgXhG8qTJ28zwKtqwauMJRPLh'],
  ['QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX',
   'QmSWUxCEPwmqkX2xYtGq355DD4pYSSsVvxVddqUidAiGXj'],
  ['QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX',
   'QmVQJbYu4gKsn52iebHFfqdnnQCDpW2o4mz4tTXk4fCgXG'],
  ['QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX',
   'QmTwiEczgjysEvJ1qu1fzBhoj1vmhwjNWCQmxPqpFPpfBr'],
  ['QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX',
   'QmTw5jkVwDJTs6rswNeitKRMyzY22BxBGdeKncu4eDy3vL'],
  ['QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX',
   'QmfNx8p4wsMVAdP5AWESCji6hBtNyEthFWw3zmSUtnMQUh'],
  ['QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn9AgJUFv8JFkwX',
   'Qmc1wWZu73HuKXTjsEdDQ5nZKTW4Xishe6F5p9nYo9dvQm'],
  ['QmcqJNJt4zxTMkB5K3Hz1VibDii47qJmn